In [2]:
import pandas as pd
import re
from tqdm.notebook import tqdm

def get_second(time_str):
    segment = time_str.split(' ')
    second = 0    
    for i in segment:
        if i[-1] == 's':
            second += int(i[:-1])
        elif i[-1] == 'm':
            second += int(i[:-1]) * 60
        elif i[-1] == 'h':
            second += int(i[:-1]) * 60 * 60
        elif i[-1] == 'd':
            second += int(i[:-1]) * 60 * 60 * 24
        else:
            print(time_str)
    return second

In [3]:
df = pd.read_csv('../dataset/raw_dataset.csv',parse_dates=['created_at'])
df['is_bot'] = df['is_bot'].astype(bool)
df.sort_values(by=['created_at'], inplace=True)

In [4]:
## create boxplot for additional cost

# Recheck [waiting time / code review time / computation time / propotion (waiting time in code review time)]
# Overall [waiting time / code review time / computation time]

tqdm.pandas()
output = dict()

df.sort_values(by=['created_at'], inplace=True)
for key, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    change_id , revision_number = key
    build_fail, recheck, build_fail_then_recheck = False, False, False 
    final_status = dict()
    prev_status = dict()
    computation_time = 0
    
    total_max_time = 0
    max_time = 0

    dict_bot = {}
    rnd = 0
    
    start_time = ''
    end_time = ''

    recheck_count = 0
    
    for ind,col in group.iterrows():
        
        time = col['created_at']
        
        if not start_time:
            start_time = time
            
        if not end_time or time > end_time:
            end_time = time
        
        #start loop if build fail is found
        if not build_fail and col['is_bot'] and 'build failed' in col['message'].lower():
            build_fail = True
            
        if not build_fail:
            continue
                                                
        if col['is_bot']:
            try:
                patch_fail = False
                build = re.findall(r'((?:merge|build) (?:succeeded|failed))',col['message'],re.IGNORECASE)
                bot = re.findall(r'((?:success|failure|aborted) in(?: \w+?)*s)',col['message'],re.IGNORECASE)
                
                if any('fail' in b.lower() for b in build): #or any('fail' in b.lower() for b in bot):
                    patch_fail = True
                elif any('abort' in b.lower() for b in build): #or any('abort' in b.lower() for b in bot):
                    patch_fail = True
                
                if build_fail and recheck:                
                    bot_time = 0
                    for i in bot:
                        bot_time += get_second(i.split(' in ')[1])
                        
                        if max_time < get_second(i.split(' in ')[1]):
                            max_time = get_second(i.split(' in ')[1])
                        
                    computation_time += bot_time
            except:
                pass
  
            if rnd not in dict_bot:
                dict_bot[rnd] = dict()

            if col['username'] not in dict_bot[rnd]:
                dict_bot[rnd][col['username']] = ''

            
            if dict_bot[rnd][col['username']] == 'failure':
                final_status[col['username']] = 'failure'
            else:
                
                if patch_fail:
                    dict_bot[rnd][col['username']] = 'failure'
                else:
                    dict_bot[rnd][col['username']] = 'success'

                final_status[col['username']] = 'failure' if patch_fail else 'success'


        
        if 'recheck' in col['message'].lower() and col['is_bot'] == False:
            recheck = True
            rnd += 1
            
            if len(final_status) > 0:
                prev_status = final_status
            final_status = dict()
            
            total_max_time += max_time
            max_time = 0
            
            if build_fail:
                build_fail_then_recheck = True
            
            recheck_count += 1
            
    total_max_time += max_time
    if build_fail_then_recheck:
        
        status = ''
        
        if len(final_status) == 0:
            final_status = prev_status
        
        for username in final_status:
            if status == 'failure':
                continue
            if final_status[username] == 'failure':
                status = 'failure'
            if final_status[username] == '':
                status = ''
            if final_status[username] == 'success':
                status = 'success'
        
        output[key] = {'computation_time': computation_time,'status': status, 'bot':dict_bot, 'waiting_time':total_max_time, 'code_review_time': (end_time - start_time).total_seconds(), 'recheck_count': recheck_count}
        

  0%|          | 0/267239 [00:00<?, ?it/s]

In [5]:
## create boxplot for additional cost

tqdm.pandas()

patch_output = dict()

df.sort_values(by=['created_at'], inplace=True)
for key, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    if key in output:
        continue
    
    change_id , revision_number = key

    computation_time = 0 
    
    recheck = False
    
    start_time = ''
    end_time = ''

    max_time = 0
    
    for ind,col in group.iterrows():
        
        time = col['created_at']
        
        if not start_time:
            start_time = time
            
        if not end_time or time > end_time:
            end_time = time
        
        if col['is_bot']:
            try:
                patch_fail = False
                build = re.findall(r'((?:merge|build) (?:succeeded|failed))',col['message'],re.IGNORECASE)
                bot = re.findall(r'((?:success|failure|aborted) in(?: \w+?)*s)',col['message'],re.IGNORECASE)
                
                if any('fail' in b.lower() for b in build): #or any('fail' in b.lower() for b in bot):
                    patch_fail = True
                elif any('abort' in b.lower() for b in build): #or any('abort' in b.lower() for b in bot):
                    patch_fail = True
                
                bot_time = 0
                for i in bot:
                    bot_time += get_second(i.split(' in ')[1])

                    if max_time < get_second(i.split(' in ')[1]):
                        max_time = get_second(i.split(' in ')[1])

                computation_time += bot_time
            except:
                pass
          
        if 'recheck' in col['message'].lower() and col['is_bot'] == False:
            
            recheck = True
            break
            
    if recheck:
        continue

    patch_output[key] = {'computation_time': computation_time, 'waiting_time':max_time, 'code_review_time': (end_time - start_time).total_seconds()}
        

  0%|          | 0/267239 [00:00<?, ?it/s]

In [6]:
output_tup = list()

dropped_case, added_case, replaced_case = 0,0,0
same_case_1, dropped_case_1, added_case_1, replaced_case_1 = 0,0,0,0

for key in tqdm(output):
    
#     proportion = round(output[key]['waiting_time'] * 100 / output[key]['code_review_time'] ,2)

    computation_time = output[key]['computation_time']
    waiting_time = output[key]['waiting_time']
    code_review_time = output[key]['code_review_time']
    status = output[key]['status']
    
    bots = output[key]['bot']
    first_bot = bots[0]
    last_bot = bots[list(bots.keys())[-1]]
    
    
    if len(bots) >= 100:
        print(key, len(bots))
    
    # scenario 
    
    # scenario #1 [Changed outcome] Justifiable
    # scenario #2 [Changed outcome] Repeated builds
    # scenario #3 [No Change] Single recheck
    # scenario #4 [No Change] Multiple recheck

    if status == 'success' and len(bots) == 2:
        scenario = 1
    elif status == 'success' and len(bots) != 2:
        scenario = 2
    elif status == 'failure' and len(bots) == 2:
        scenario = 3
    elif status == 'failure' and len(bots) != 2:
        scenario = 4
    else:
        print("should not be here", key)
    
    bot_modify = 0
    
    # l_key union f_key
    f_u_l = set(first_bot.keys()).union(set(last_bot.keys()))
    # cehck if L == F
    if set(first_bot.keys()) == set(last_bot.keys()):
        bot_modify = 'same bot'
    # check if L in F, FuL == F [dropped case]
    elif set(first_bot.keys()) == f_u_l:
        bot_modify = 'drop bot'
    # check if F in L, FuL == L [added case]
    elif set(last_bot.keys()) == f_u_l:
        bot_modify = 'add bot'
    # replace case
    else:
        bot_modify = 'replace bot'
    
    output_tup.append((key, scenario, bot_modify, status, computation_time, waiting_time, code_review_time))
recheck_df = pd.DataFrame(output_tup, columns=['key', 'scenario', 'bot_modify', 'status', 'computation_time', 'waiting_time', 'code_review_time'])

  0%|          | 0/36783 [00:00<?, ?it/s]

('I9a5b48d58ff9a1ba651468b818cc669e1ca099ff', 3) 116
('Ibea13629fafc40ffd9ba64a318dcfd691c88a1dc', 1) 175


In [6]:
## - Break down result from table 3,4 for multiple recheck with change outcome 
##   to the first chunk unchange outcome[x hours] change outcome[x hours]

scenario_2_df = recheck_df[recheck_df['scenario'] == 2].copy()
scenario_2_df.reset_index(inplace=True)
scenario_2_keys = scenario_2_df['key'].tolist()

for key, group in tqdm(df.groupby(['change_id','revision_number'])):    
    
    if key not in scenario_2_keys:
        continue
    
    scenario_2_index = scenario_2_keys.index(key)
    
    computation_time = 0
    max_time = 0
    prev_computation_time = 0
    prev_max_time = 0

    for ind,col in group.iterrows():
        
        #start loop if build fail is found
        if not build_fail and col['is_bot'] and 'build' in col['message'].lower():
            build_fail = True
            
        if not build_fail:
            continue
                                                
        if col['is_bot']:
            try:
                patch_fail = False
                build = re.findall(r'((?:merge|build) (?:succeeded|failed))',col['message'],re.IGNORECASE)
                bot = re.findall(r'((?:success|failure|aborted) in(?: \w+?)*s)',col['message'],re.IGNORECASE)
                
                if any('fail' in b.lower() for b in build): #or any('fail' in b.lower() for b in bot):
                    patch_fail = True
                elif any('abort' in b.lower() for b in build): #or any('abort' in b.lower() for b in bot):
                    patch_fail = True
                
                if build_fail and recheck:                
                    bot_time = 0
                    for i in bot:
                        bot_time += get_second(i.split(' in ')[1])
                        
                        if max_time < get_second(i.split(' in ')[1]):
                            max_time = get_second(i.split(' in ')[1])
                        
                    computation_time += bot_time
            except:
                pass
  
        
        if 'recheck' in col['message'].lower() and col['is_bot'] == False:
            recheck = True
            if computation_time and max_time:
                prev_computation_time = computation_time
                prev_max_time = max_time
            
            max_time = 0
            computation_time = 0
            
    if max_time == 0:
        max_time = prev_max_time
        computation_time = prev_computation_time
        
    scenario_2_df.at[scenario_2_index, 'last_recheck_computation_time'] = computation_time
    scenario_2_df.at[scenario_2_index, 'last_recheck_waiting_time'] = max_time

  0%|          | 0/267239 [00:00<?, ?it/s]

In [8]:
from datetime import timedelta

print(scenario_2_df.shape[0])

for i in ['computation_time', 'waiting_time']:
    total_time = int(scenario_2_df[i].sum())
    last_time = int(scenario_2_df["last_recheck_" + i].sum())
    accu_time = total_time - last_time
    print(f'total    {i}: {timedelta(seconds=total_time)}')
    print(f'accumerate {i} before change CI outcome: {timedelta(seconds=accu_time)}, ({round(accu_time * 100 / total_time)}%)')
    print(f'last recheck {i}: {timedelta(seconds=last_time)}, ({round(last_time * 100 / total_time)}%)')
    print('-------------')

6517
total    computation_time: 22836 days, 14:33:00
accumerate computation_time before change CI outcome: 15888 days, 18:26:55, (70%)
last recheck computation_time: 6947 days, 20:06:05, (30%)
-------------
total    waiting_time: 1890 days, 4:30:25
accumerate waiting_time before change CI outcome: 1361 days, 14:24:14, (72%)
last recheck waiting_time: 528 days, 14:06:11, (28%)
-------------


In [21]:
## As suggested in the Intro, I think we should not compare against a normal code review for 
## Observation 7 and 8. However, we can calculate the proportion of waiting time to the total review time 
## per code review. Then, we can discuss that "what if unjustifiable rechecks are not invoked we can save X% 
## of review time and computation resources"

## observation 7,8 calculate total computation time from scenario 2,3,4 
## then calculate time before recheck [both waiting and computation] and after recheck
## both computation_time and waiting_time after recheck already compute then we just calculate before recheck
cost_comparison_df = recheck_df.copy()
cost_comparison_df.reset_index(inplace=True)
cost_comparison_keys = cost_comparison_df['key'].tolist()

for key, group in tqdm(df.groupby(['change_id','revision_number'])):    
    
    if key not in cost_comparison_keys:
        continue
    
    cost_comparison_index = cost_comparison_keys.index(key)
    
    computation_time = 0
    max_time = 0
    start_time = ''
    end_time = ''

    for ind,col in group.iterrows():
        
        if col['is_bot']:
            try:
                bot = re.findall(r'((?:success|failure|aborted) in(?: \w+?)*s)',col['message'],re.IGNORECASE)
                bot_time = 0
                for i in bot:
                    bot_time += get_second(i.split(' in ')[1])
                    if max_time < get_second(i.split(' in ')[1]):
                        max_time = get_second(i.split(' in ')[1])
                computation_time += bot_time
            except:
                pass

        if 'recheck' in col['message'].lower() and col['is_bot'] == False:
            break
        
        time = col['created_at']
                
        if not start_time:
            start_time = time
            
        if not end_time or time > end_time:
            end_time = time

                
    cost_comparison_df.at[cost_comparison_index, 'before_recheck_computation_time'] = computation_time
    cost_comparison_df.at[cost_comparison_index, 'before_recheck_waiting_time'] = max_time
    try:
        cost_comparison_df.at[cost_comparison_index, 'before_recheck_code_review_time'] = (end_time - start_time).total_seconds()
    except:
        cost_comparison_df.at[cost_comparison_index, 'before_recheck_code_review_time'] = 0

  0%|          | 0/267239 [00:00<?, ?it/s]

In [31]:
import numpy as np
## calculate percentage computation / review time waste generate compare to before recheck case


for ind, col in tqdm(cost_comparison_df.iterrows(),total=cost_comparison_df.shape[0]):
    
    try:
        computation_percent = round((col['before_recheck_computation_time'] * 100) / (col['before_recheck_computation_time'] + col['computation_time']))
    except:
        computation_percent = np.nan
        
    try:
        code_review_percent = round((col['before_recheck_code_review_time'] * 100) / (col['before_recheck_code_review_time'] + col['code_review_time']))
    except:
        code_review_percent = np.nan
        
    cost_comparison_df.at[ind, 'computation_percent'] = computation_percent
    cost_comparison_df.at[ind, 'code_review_percent'] = code_review_percent


  0%|          | 0/36783 [00:00<?, ?it/s]

In [87]:
for scenario in range(1,5):
    print(f'compution percent {scenario}:',cost_comparison_df[cost_comparison_df['scenario'] == scenario]['computation_percent'].median())
    print(f'code review percent {scenario}:',cost_comparison_df[cost_comparison_df['scenario'] == scenario]['code_review_percent'].median())
    
print(f'compution percent all:',cost_comparison_df[cost_comparison_df['scenario'] == scenario]['computation_percent'].median())
print(f'code review percent all:',cost_comparison_df[cost_comparison_df['scenario'] == scenario]['code_review_percent'].median())
    
print(f'compution percent unjustified:',cost_comparison_df[cost_comparison_df['scenario'].isin([2,3,4])]['computation_percent'].median())
print(f'code review percent unjustified:',cost_comparison_df[cost_comparison_df['scenario'].isin([2,3,4])]['code_review_percent'].median())

compution percent 1: 48.0
code review percent 1: 11.0
compution percent 2: 25.0
code review percent 2: 5.0
compution percent 3: 49.0
code review percent 3: 11.0
compution percent 4: 31.0
code review percent 4: 6.0
compution percent all: 31.0
code review percent all: 6.0
compution percent unjustified: 37.0
code review percent unjustified: 8.0


In [71]:
print(len(patch_output))
patch_tup = list()

for key in patch_output:    
    patch_tup.append((patch_output[key]['computation_time'], patch_output[key]['waiting_time'], patch_output[key]['code_review_time']))
patch_df = pd.DataFrame(patch_tup, columns=['computation_time', 'waiting_time', 'code_review_time'])

patch_df['scenario'] = 'without_recheck'
print(patch_df['computation_time'].mean())
print(patch_df['code_review_time'].mean())                    

225863
41707.98268862098
1093104.9983264192


In [ ]:
## try rain cloud

import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
#sns.set(style="darkgrid")
#sns.set(style="whitegrid")
#sns.set_style("white")
sns.set(style="whitegrid",font_scale=2)
import matplotlib.collections as clt
import ptitprince as pt

In [ ]:
# computation_time

import seaborn as sns
import matplotlib.pyplot as plt

from statannotations.Annotator import Annotator
sns.set(style="whitegrid")

f, ax = plt.subplots(4,1,sharex=True,figsize=(4,6))

ort = "h"; pal = "Set2"; sigma = .2
time = 'computation_time'

# plot_df = recheck_df[['scenario',time]].copy()
# plot_df['scenario'] = 'All'

plot_df = pd.DataFrame()
for i in range(1,5):
    temp_df = recheck_df[recheck_df['scenario'] == i][['scenario', time]]
    plot_df = pd.concat([plot_df, temp_df], ignore_index=True)

plot_df['scenario'] = plot_df['scenario'].astype(str)
    
for i in range(len(plot_df['scenario'].unique())):
    name_list = plot_df['scenario'].unique()               
    temp_df = plot_df[plot_df['scenario'] == name_list[i]]
    temp_df = pd.concat([patch_df[['scenario', time]], temp_df], ignore_index=True)
    temp_df[time] = temp_df[time] / 3600
    
    #     swarm_plot = sns.boxplot(ax=ax[i], data=temp_df, y='scenario', x=time, showfliers = False)
        
    swarm_plot=pt.half_violinplot(ax=ax[i], data=temp_df, y='scenario', x=time, palette = pal, bw = .2, cut = 0.,
                          scale = "area", width = .6, inner = None, orient = ort)
    swarm_plot=sns.stripplot(ax=ax[i], data=temp_df, y='scenario', x=time, palette = pal, edgecolor = "white",
                     size = 3, jitter = 1, zorder = 0, orient = ort,)
    swarm_plot=sns.boxplot(ax=ax[i], data=temp_df, y='scenario', x=time, color = "black", width = .15, zorder = 10,\
                showcaps = True, boxprops = {'facecolor':'none', "zorder":10},\
                showfliers=False, whiskerprops = {'linewidth':2, "zorder":10},\
                   saturation = 1, orient = ort)
    
    label_title = ['Justifiable', 'Changed outcome multiple rechecks', 'Unchanged outcome single recheck', 'Unchanged outcome multiple rechecks']
    swarm_plot.set_yticklabels(['Without recheck', label_title[i]])
    
    ax[i].set(xscale="log")
    ax[i].set_xlabel("")
    ax[i].set_ylabel("")    
    
ax[3].set_xlabel("Durations (log hr)")    
# put stats in graph

for i in range(len(plot_df['scenario'].unique())):
    name_list = plot_df['scenario'].unique()
    temp_df = plot_df[plot_df['scenario'] == name_list[i]]
    temp_df = pd.concat([patch_df[['scenario', time]], temp_df], ignore_index=True)
    temp_df[time] = temp_df[time] / 3600

    pairs = [('without_recheck',name_list[i])]
    
    annot = Annotator(ax=ax[i], pairs=pairs, data=temp_df, y='scenario', x=time,orient='h')
    annot.configure(test='Mann-Whitney', text_format='star', loc='outside')
    annot.apply_test().annotate()

fig = swarm_plot.get_figure()
fig.savefig(f'../fig/RQ3_{time}.pdf', format='pdf', bbox_inches="tight")

In [ ]:
# code_review_time

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

f, ax = plt.subplots(4,1,sharex=True,figsize=(4,6))
ax = ax.flatten()
# f.delaxes(ax[5])

ort = "h"; pal = "Set2"; sigma = .2
time = 'code_review_time'

plot_df = pd.DataFrame()
for i in range(1,5):
    temp_df = recheck_df[recheck_df['scenario'] == i][['scenario', time]]
    plot_df = pd.concat([plot_df, temp_df], ignore_index=True)
    
plot_df['scenario'] = plot_df['scenario'].astype(str)

for i in range(len(plot_df['scenario'].unique())):
        
    name_list = plot_df['scenario'].unique()               
    temp_df = plot_df[plot_df['scenario'] == name_list[i]]
    temp_df = pd.concat([patch_df[['scenario', time]], temp_df], ignore_index=True)
    
    temp_df[time] = temp_df[time] / 3600
        
    swarm_plot=pt.half_violinplot(ax=ax[i], data=temp_df, y='scenario', x=time, palette = pal, bw = .2, cut = 0.,
                          scale = "area", width = .6, inner = None, orient = ort)
    swarm_plot=sns.stripplot(ax=ax[i], data=temp_df, y='scenario', x=time, palette = pal, edgecolor = "white",
                     size = 3, jitter = 1, zorder = 0, orient = ort,)
    swarm_plot=sns.boxplot(ax=ax[i], data=temp_df, y='scenario', x=time, color = "black", width = .15, zorder = 10,\
                showcaps = True, boxprops = {'facecolor':'none', "zorder":10},\
                showfliers=False, whiskerprops = {'linewidth':2, "zorder":10},\
                   saturation = 1, orient = ort)
    
    label_title = ['Justifiable', 'Changed outcome multiple rechecks', 'Unchanged outcome single recheck', 'Unchanged outcome multiple rechecks']
    swarm_plot.set_yticklabels(['Without recheck', label_title[i]])
    ax[i].set(xscale="log")
               
    ax[i].set_xlabel("")
    ax[i].set_ylabel("")

ax[3].set_xlabel("Durations (log hr)")    

# put stats in graph

for i in range(len(plot_df['scenario'].unique())):
    name_list = plot_df['scenario'].unique()
    temp_df = plot_df[plot_df['scenario'] == name_list[i]]
    temp_df = pd.concat([patch_df[['scenario', time]], temp_df], ignore_index=True)
    temp_df[time] = temp_df[time] / 3600

    pairs = [('without_recheck',name_list[i])]
    
    annot = Annotator(ax=ax[i], pairs=pairs, data=temp_df, y='scenario', x=time,orient='h')
    annot.configure(test='Mann-Whitney', text_format='star', loc='outside')
    annot.apply_test().annotate()

fig = swarm_plot.get_figure()
fig.savefig(f'../fig/RQ3_{time}.pdf', format='pdf', bbox_inches="tight")

In [ ]:
## criff-delta

from cliffs_delta import cliffs_delta

for scenario in sorted(recheck_df['scenario'].unique()):
    scenario_df = recheck_df[recheck_df['scenario'] == scenario]
    
    d, res = cliffs_delta(scenario_df['code_review_time'], patch_df['code_review_time'])
    print("code_review_time:",scenario, d, res)
    d, res = cliffs_delta(scenario_df['computation_time'], patch_df['computation_time'])
    print("computation_time:",scenario, d, res)


In [ ]:
## generate table V, VI

from datetime import timedelta

for scenario in sorted(recheck_df['scenario'].unique()):
    scenario_df = recheck_df[recheck_df['scenario'] == scenario]
    
    print(f'code_review_time: {scenario}', timedelta(seconds=int(scenario_df['code_review_time'].median())))

print(f'code_review_time: no recheck', timedelta(seconds=int(patch_df['code_review_time'].median())))
print(f'code_review_time: all', timedelta(seconds=int(recheck_df['code_review_time'].median())))

for scenario in sorted(recheck_df['scenario'].unique()):
    scenario_df = recheck_df[recheck_df['scenario'] == scenario]
    print(f'computation_time: {scenario}', timedelta(seconds=int(scenario_df['computation_time'].median())))

print(f'computation_time: no recheck', timedelta(seconds=int(patch_df['computation_time'].median())))
print(f'computation_time: all', timedelta(seconds=int(recheck_df['computation_time'].median())))

for scenario in sorted(recheck_df['scenario'].unique()):
    scenario_df = recheck_df[recheck_df['scenario'] == scenario]
    print(f'waiting_time: {scenario}', timedelta(seconds=int(scenario_df['waiting_time'].median())))
print(f'waiting_time: all', timedelta(seconds=int(recheck_df['waiting_time'].median())))

In [ ]:
from datetime import timedelta
# computation time

for i in range(1,5):
    temp_df = recheck_df[(recheck_df['scenario'] == i)]
    print(f'computation time {i}: {timedelta(seconds=int(temp_df["computation_time"].sum()))}')

print(f'computation time All: {timedelta(seconds=int(recheck_df["computation_time"].sum()))}')
        
# waiting time

for i in range(1,5):
    temp_df = recheck_df[(recheck_df['scenario'] == i)]    
    print(f'waiting time {i}: {timedelta(seconds=int(temp_df["waiting_time"].sum()))}')

print(f'waiting time All: {timedelta(seconds=int(recheck_df["waiting_time"].sum()))}')

In [ ]:
print(f'waiting time All: {timedelta(seconds=int(patch_df["computation_time"].sum()))}')
print(f'waiting time All: {timedelta(seconds=int(patch_df["code_review_time"].sum()))}')
